In [1]:
import json
from collections import Counter
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys
import os
import argparse
from os.path import dirname, realpath

import hashlib
import datetime
import datetime as datetime
import sklearn
import statistics
sys.path.append("./src/")
#from disrisknet.utils.parsing import parse_args
import pdb

src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()

from sklearn.metrics import confusion_matrix

In [4]:
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/allChars_DF.csv'))
from sklearn.metrics import confusion_matrix


In [5]:
logpath = os.path.join(src_path ,'logs_transformer_vte/F/Ran10_20/', "af7e89f5c48255bfb508b07c4b2638c6.results.test_preds")


In [6]:
 
with open(logpath, 'rb') as f:
    R6 = pickle.load(f)
    p6 = np.array(R6['probs'])

pat_ids = (testDF['patient_id'] ).astype(int)
 

In [7]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [8]:
p = p6[:,1]

In [9]:
testDF['pids'] = pat_ids.apply(md5)

g6 = pd.DataFrame({'p6': np.array(R6['probs'])[:,1] ,    
                   'pids':np.array(R6['pids']) ,    
                   'model_date':np.array(R6['dates'])
                  })

In [14]:
M = pd.merge(g6, testDF)
m = M[["patient_id","dx_date", "index_date", "model_date",  "outcome","outcome_date","p6" ]]
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
m1 = m

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_76000\3003564994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])


In [10]:
M = pd.merge(g6, testDF)
m = M[["patient_id","dx_date", "index_date", "model_date",  "outcome","outcome_date","p6" ]]
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_76000\3732438356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_76000\3732438356.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_76000\3732438356.py:13: SettingWithCopyWarning: 
A value is trying to be

In [15]:
sklearn.metrics.roc_auc_score(m1['outcome'],  m1['p6'], average = 'samples')


0.6536828283210259

In [16]:
( confusion_matrix (m1['outcome'],  m1['p6']>.5) )

array([[21845,    62],
       [ 2245,   541]], dtype=int64)

In [17]:
# how many people had event within 6 months of index date

In [18]:
m1.to_csv('output/Co5_Dates.csv') 

In [8]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [27]:
statistics.quantiles( M['len180'],n=20)

[289.0,
 390.0,
 471.0,
 541.0,
 610.0,
 677.0,
 743.0,
 809.0,
 879.0,
 952.0,
 1036.0,
 1128.0,
 1230.0,
 1343.0,
 1482.5,
 1647.0,
 1880.9,
 2189.0,
 2772.0]

In [23]:
plt.hist(M['len180'])

(array([2.2715e+04, 1.8050e+03, 1.3000e+02, 2.5000e+01, 1.2000e+01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.0000e+00]),
 array([1.00000e+00, 2.37810e+03, 4.75520e+03, 7.13230e+03, 9.50940e+03,
        1.18865e+04, 1.42636e+04, 1.66407e+04, 1.90178e+04, 2.13949e+04,
        2.37720e+04]),
 <BarContainer object of 10 artists>)